In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from __future__ import print_function
import tensorflow as tf
#from tensorflow import keras

import keras
from keras import layers

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

/Users/anthony/anaconda3/envs/ImgCV/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
# import mnist
from keras.datasets import mnist

(x_train_raw, y_train_raw), (x_test_raw, y_test_raw) = mnist.load_data()

print(np.shape(x_train_raw))
print(np.shape(y_train_raw))
print(len(set(y_train_raw)))

(60000, 28, 28)
(60000,)
10


In [13]:
# Get labels and encode
from sklearn.preprocessing import OneHotEncoder

# reshape vectors
y_train_raw = y_train_raw.reshape(len(y_train_raw), 1)
y_test_raw = y_test_raw.reshape(len(y_test_raw), 1)


# One hot encode
encoder_OH = OneHotEncoder(sparse=False)

y_preShuf_train = encoder_OH.fit_transform(y_train_raw)
y_preShuf_test = encoder_OH.fit_transform(y_test_raw)

# Shuffle
from sklearn.utils import shuffle
X_train_full, y_train_full = shuffle(x_train_raw, y_preShuf_train, random_state=42)
X_test, y_test = shuffle(x_test_raw, y_preShuf_test)

# Extend the dimension to RGB dimension
X_train_full = np.expand_dims(X_train_full, axis=3)
X_test = np.expand_dims(X_test, axis=3)

print(np.shape(X_train))

(60000, 28, 28, 1)


In [17]:
# take subset of X train

N_SUB = 3000

X_train_sub = X_train_full[0:N_SUB]
y_train_sub = y_train_full[0:N_SUB]

print(np.shape(X_train_sub))

(3000, 28, 28, 1)


### Define model

In [18]:
def leNet4():
    # create model
    model = Sequential()
    
    # Convolution
    model.add(Conv2D(4, (4, 4), input_shape=(28,28,1), activation='relu'))
    
    # Subsampling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.1))
    
    # 2nd convolution layer
    model.add( Conv2D(16, (3, 3), activation='relu'))
    
    # Subsampling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.1))
    
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dropout(0.4))
    
    #model.add(Dense(128, activation='relu'), )
    #model.add(Dropout(0.4))
    
    #model.add(Dense(40, activation='relu'), )
    #model.add(Dropout(0.2))
    
    model.add(Dense(10, activation='softmax'))
    
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    return model 

In [19]:
model = leNet4()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 25, 25, 4)         68        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 4)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 16)        592       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 120)               48120     
_________________________________________________________________
dropout_3 (Dropout)          (None, 120)               0         
__________

In [22]:
history = model.fit(X_train_sub, y_train_sub,
                    epochs=10,
                    verbose=1,
                    batch_size=100,
                    validation_data=(X_test, y_test)
                   )

Train on 3000 samples, validate on 10000 samples
Epoch 1/10
3000/3000 [==============================] - 2s 738us/step - loss: 0.4247 - acc: 0.8650 - val_loss: 0.2991 - val_acc: 0.9145
Epoch 2/10
3000/3000 [==============================] - 2s 721us/step - loss: 0.3613 - acc: 0.8823 - val_loss: 0.2722 - val_acc: 0.9190
Epoch 3/10
3000/3000 [==============================] - 2s 724us/step - loss: 0.3196 - acc: 0.8977 - val_loss: 0.2690 - val_acc: 0.9215
Epoch 4/10
3000/3000 [==============================] - 2s 735us/step - loss: 0.2792 - acc: 0.9147 - val_loss: 0.2465 - val_acc: 0.9293
Epoch 5/10
3000/3000 [==============================] - 2s 723us/step - loss: 0.2458 - acc: 0.9220 - val_loss: 0.2439 - val_acc: 0.9305
Epoch 6/10
3000/3000 [==============================] - 2s 746us/step - loss: 0.2285 - acc: 0.9293 - val_loss: 0.2271 - val_acc: 0.9358
Epoch 7/10
3000/3000 [==============================] - 2s 746us/step - loss: 0.1888 - acc: 0.9377 - val_loss: 0.2245 - val_acc: 0.9391